In [1]:
import boto3
import io
import zipfile
from io import BytesIO

In [2]:
s3 = boto3.resource("s3",
                    endpoint_url = "http://192.168.0.29",
                    aws_access_key_id="AKIAPo19vPR_TJaeVgleCiOSUw",
                    aws_secret_access_key="7cSWM1KCXvRpK4ICeDEAfuicEm+QQeuhqOi7cejZ",
                    region_name = 'eu-central-1',
               )

In [3]:
s3_client = boto3.client("s3",
                    endpoint_url = "http://192.168.0.29",
                    aws_access_key_id="AKIAPo19vPR_TJaeVgleCiOSUw",
                    aws_secret_access_key="7cSWM1KCXvRpK4ICeDEAfuicEm+QQeuhqOi7cejZ",
                   )
               

In [4]:
### create bucket
def create_bucket(bucket_name, region=None):
    if not s3.Bucket(bucket_name) in s3.buckets.all():
        if region is None:
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
    else:
        print("Bucket name already exists")


In [5]:
## list all buckets in the s3 a/c
def list_bucket():
    buckets_list = s3_client.list_buckets()
    for bucket in buckets_list['Buckets']:
        print(bucket)
    return True


In [6]:
#delete s3 bucket
def delete_bucket(bucket_name):
    bucket = s3.Bucket(bucket_name)
    for key in bucket.objects.all():
        key.delete()
    bucket.delete()

In [7]:

def upload_file(file_name, bucket, object_name=None):
    if s3_client.get_object(Bucket = bucket, Key = file_name):
        print("File already uploaded")
    else:
        if object_name is None:
            object_name = file_name

        # Upload the file
        try:
            response = s3_client.upload_file(file_name, bucket, object_name)
        except ClientError as e:
            logging.error(e)
            return False
        return True

In [8]:
create_bucket("saved-models")

Bucket name already exists


In [9]:
list_bucket()

{'Name': 'agromodel', 'CreationDate': datetime.datetime(2021, 4, 22, 8, 37, 13, tzinfo=tzutc())}
{'Name': 'sample-dataset', 'CreationDate': datetime.datetime(2021, 4, 24, 7, 44, 47, tzinfo=tzutc())}
{'Name': 'saved-models', 'CreationDate': datetime.datetime(2021, 4, 26, 14, 1, 40, tzinfo=tzutc())}
{'Name': 'split-dataset', 'CreationDate': datetime.datetime(2021, 4, 23, 21, 8, 10, tzinfo=tzutc())}


True

In [10]:
upload_file("sample_data.zip", "sample-dataset")

File already uploaded


#### Unzip the dataset on s3
- Unzip sample_data.zip contents to the  Cortx-s3 bucket.

- I have already run this process previously, therefore going to skip.....

In [ ]:
bucket = 'sample-dataset'

prefix      = "sample_data.zip"
zipped_keys =  s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix)
file_list = []
for key in zipped_keys['Contents']:
    file_list.append(key['Key'])

zip_obj = s3.Object(bucket_name=bucket, key=file_list[0])
buffer = BytesIO(zip_obj.get()["Body"].read())


z = zipfile.ZipFile(buffer)
for filename in z.namelist():
    file_info = z.getinfo(filename)
    s3.meta.client.upload_fileobj(
        z.open(filename),
        Bucket=bucket,
        Key=filename)

#### Split the data into train and validation sets
- Run this ONLY when copyobject is implemeted on Cortx-S3

- The initial idea was to upload the image dataset and then split it on the Cortx-S3 using the methods below. However, some functions like move and copy file are not implemented on the current build of s3. Therefore I choose to upload the dataset that is already split. In future you can choose to use the method below to the split data.

In [ ]:
import pathlib
import random
import shutil
from os import path

train_path = "s3://agromodel/Plant_leave_diseases_dataset_without_augmentation/"
def list_dirs(directory):
    """Returns all directories in a train directory
    """
    return [folder["name"] for folder in client.listdir(directory)[1:]]


def list_files(directory):
    """Returns all the files in a directory
    """
    return [file for file in client.ls(directory)]


def split_ratio(input_data, seed, ratio=(0.8, 0.2)):
    assert sum(ratio) == 1
    for cls_folder in list_dirs(input_data):
        split_cls_directory(cls_folder, ratio, seed)


def split_cls_directory(directory, ratio, seed):
    random.seed(seed)
    files = list_files(directory)
    random.shuffle(files)

    split_train = int(ratio[0] * len(files))

    data = split_files(files, split_train)
    copy_files(data, directory)


def split_files(files, split_train):
    files_train = files[:split_train]
    files_val = files[split_train:]

    data = [(files_train, 'agromodel/train'), (files_val, 'agromodel/val')]
    return data


def copy_files(data, directory):
    """Copy the files to the respective folders"""
    class_name = path.split(directory)[1]
    for (files, data_type) in data:
        print(data_type)
        full_path = path.join(data_type, class_name)
        print("path", full_path)
        client.mkdir("s3://" + full_path, exist_ok=True)
        for file in files:
            client.copy(file, "s3://" + full_path)


x = split_ratio(train_path, ratio=(0.8, 0.2), seed=42)
